In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import os
import seaborn as sns
import random 
import scipy as sc
import statsmodels.api as sm
import scipy.stats as stats
import cartopy.crs as ccrs
import pygrib
import cfgrib
import stormeunice as eun

sns.set_theme(style="white")
sns.set_style("white")

random.seed(10)

# Import data

In [2]:
directory, experiments, inits, cfpf = eun.data.Data.load_meta()
lat, lon = eun.data.Data.get_latlon()
south_df = eun.data.Data.get_friday_data()

# Stamp plot differences